In [2]:
import argparse
import json
import sys
from pathlib import Path
from types import SimpleNamespace

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

In [3]:
def ejecutar_notebook_y_extraer_vars(nb_path: Path) -> dict:
    """
    Ejecuta secuencialmente las celdas de código del notebook dado y devuelve
    un diccionario con las variables definidas (namespace).

    No modifica el .ipynb original.
    """
    import nbformat, runpy

    nb = nbformat.read(str(nb_path), as_version=4)
    # Ambiente aislado donde se ejecutarán las celdas
    ns = {}
    # Ejecutar solo celdas de código en orden
    for cell in nb.cells:
        if cell.cell_type == "code":
            code = cell.source
            if code.strip():
                exec(compile(code, filename="<notebook_cell>", mode="exec"), ns, ns)
    return ns


In [4]:
def extraer_Xy_desde_namespace(ns: dict):
    """
    Intenta recuperar los DataFrames preprocesados del notebook:
      - train_scaled (features estandarizadas por día, con columna 'date' y quizás 'Close', etc.)
      - train_df     (debe contener columna 'signal' con {-1,0,1})
    Construye X (features) e y (etiquetas) alineados por índice temporal.
    """
    if "train_scaled" not in ns:
        raise RuntimeError("No se encontró 'train_scaled' en el notebook. Asegúrate de ejecutar feature_eng.ipynb antes.")
    if "train_df" not in ns:
        raise RuntimeError("No se encontró 'train_df' en el notebook. Asegúrate de ejecutar feature_eng.ipynb antes.")

    train_scaled = ns["train_scaled"].copy()
    train_df = ns["train_df"].copy()

    # Alinear por índice (suponemos orden temporal ya preservado en el notebook)
    df_merged = train_scaled.join(train_df[["signal"]], how="inner")
    # Features = todo salvo 'date' y 'signal'
    X = df_merged.drop(columns=[c for c in ["date", "signal"] if c in df_merged.columns], errors="ignore").values.astype(np.float32)
    y_signal = df_merged["signal"].astype(int).values  # {-1,0,1}

    # Mapear a 0..K-1 para Keras
    clases_unicas = sorted(np.unique(y_signal).tolist())
    clase2idx = {c: i for i, c in enumerate(clases_unicas)}
    idx2clase = {i: c for c, i in clase2idx.items()}
    y = np.array([clase2idx[c] for c in y_signal], dtype=np.int64)

    return SimpleNamespace(
        X=X,
        y=y,
        clases_unicas=clases_unicas,
        clase2idx=clase2idx,
        idx2clase=idx2clase,
        train_scaled=train_scaled,
        train_df=train_df,
    )

In [5]:
def split_temporal(X, y, val_ratio=0.15, test_ratio=0.15):
    """
    Split temporal: primeros registros -> train, luego val y test.
    """
    n = len(X)
    n_test = int(n * test_ratio)
    n_val = int(n * val_ratio)
    n_train = n - n_val - n_test
    return (X[:n_train], y[:n_train],
            X[n_train:n_train+n_val], y[n_train:n_train+n_val],
            X[n_train+n_val:], y[n_train+n_val:])

In [6]:
def preparar_class_weights(y_train):
    clases = np.unique(y_train)
    pesos = compute_class_weight(class_weight="balanced", classes=clases, y=y_train)
    return {int(c): float(w) for c, w in zip(clases, pesos)}

In [8]:
def construir_mlp(n_features, n_clases, hidden=(128, 64), dropout=0.2, lr=1e-3):
    inputs = keras.Input(shape=(n_features,))
    x = inputs
    for h in hidden:
        x = layers.Dense(h, activation="relu")(x)
        x = layers.Dropout(dropout)(x)
        x = layers.BatchNormalization()(x)
    outputs = layers.Dense(n_clases, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="MLP_Baseline")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [9]:
def construir_ventanas(X, y, window=32, horizon=1, step=1):
    """
    Convierte series diarias X (N, F) en secuencias para CNN: (M, window, F)
    La etiqueta es la del tiempo t + horizon en el extremo de la ventana.
    """
    N = len(X)
    fin = N - window - horizon + 1
    if fin <= 0:
        raise ValueError("Datos insuficientes para la ventana/horizonte.")
    X_seq, y_seq = [], []
    for i in range(0, fin, step):
        X_seq.append(X[i:i+window])
        y_seq.append(y[i+window-1 + horizon])
    return np.asarray(X_seq, dtype=np.float32), np.asarray(y_seq, dtype=np.int64)

In [10]:
def construir_cnn_1d(window, n_features, n_clases,
                     filtros=(64, 128), kernel_size=3, pool_size=2, dropout=0.3, lr=1e-3):
    inputs = keras.Input(shape=(window, n_features))
    x = inputs
    for f in filtros:
        x = layers.Conv1D(filters=f, kernel_size=kernel_size, padding="causal", activation="relu")(x)
        x = layers.MaxPooling1D(pool_size=pool_size)(x)
        x = layers.Dropout(dropout)(x)
        x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(n_clases, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="CNN1D_Temporal")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [11]:
def evaluar(model, X_test, y_test, idx2clase):
    proba = model.predict(X_test, verbose=0)
    y_pred = np.argmax(proba, axis=1)
    target_names = [str(idx2clase[i]) for i in sorted(idx2clase.keys())]
    print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))
    print("\nReporte de clasificación:\n", classification_report(y_test, y_pred, target_names=target_names, digits=4))

In [12]:
def entrenar_mlp(ns_data, epochs=200, batch_size=256):
    X, y = ns_data.X, ns_data.y
    Xtr, ytr, Xva, yva, Xte, yte = split_temporal(X, y, val_ratio=0.15, test_ratio=0.15)
    cw = preparar_class_weights(ytr)
    model = construir_mlp(n_features=X.shape[1], n_clases=len(np.unique(y)))
    callbacks = [
        keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor="val_loss"),
        keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-5),
    ]
    model.fit(Xtr, ytr, validation_data=(Xva, yva),
              epochs=epochs, batch_size=batch_size, class_weight=cw, callbacks=callbacks, verbose=1)
    evaluar(model, Xte, yte, ns_data.idx2clase)
    model.save("mlp_baseline.keras")
    print("✔ Guardado: mlp_baseline.keras")
    return model

In [13]:
def entrenar_cnn(ns_data, window=32, horizon=1, epochs=200, batch_size=256):
    X, y = ns_data.X, ns_data.y
    # Ventanas sobre TODO el histórico en escala ya estandarizada
    X_seq, y_seq = construir_ventanas(X, y, window=window, horizon=horizon, step=1)
    n = len(X_seq)
    n_test = int(n * 0.15)
    n_val = int(n * 0.15)
    n_train = n - n_val - n_test
    Xtr, ytr = X_seq[:n_train], y_seq[:n_train]
    Xva, yva = X_seq[n_train:n_train+n_val], y_seq[n_train:n_train+n_val]
    Xte, yte = X_seq[n_train+n_val:], y_seq[n_train+n_val:]

    cw = preparar_class_weights(ytr)
    model = construir_cnn_1d(window=window, n_features=X.shape[1], n_clases=len(np.unique(y)))
    callbacks = [
        keras.callbacks.EarlyStopping(patience=12, restore_best_weights=True, monitor="val_loss"),
        keras.callbacks.ReduceLROnPlateau(patience=6, factor=0.5, min_lr=1e-5),
    ]
    model.fit(Xtr, ytr, validation_data=(Xva, yva),
              epochs=epochs, batch_size=batch_size, class_weight=cw, callbacks=callbacks, verbose=1)
    evaluar(model, Xte, yte, ns_data.idx2clase)
    model.save("cnn_temporal.keras")
    print("✔ Guardado: cnn_temporal.keras")
    return model

In [15]:
def main():
    parser = argparse.ArgumentParser(description="Continuación DL: MLP y CNN a partir de feature_eng.ipynb (sin modificarlo)")
    parser.add_argument("--nb", required=True, help="Ruta al feature_eng.ipynb")
    parser.add_argument("--modelo", choices=["mlp", "cnn"], required=True, help="Modelo a entrenar")
    parser.add_argument("--window", type=int, default=32, help="(CNN) Tamaño de ventana")
    parser.add_argument("--horizon", type=int, default=1, help="(CNN) Horizonte de predicción")
    parser.add_argument("--epochs", type=int, default=200, help="Épocas de entrenamiento")
    parser.add_argument("--batch-size", type=int, default=256, help="Tamaño de lote")
    args = parser.parse_args()

    nb_path = Path(args.nb)
    if not nb_path.exists():
        print(f"[ERROR] No se encontró el notebook en: {nb_path}", file=sys.stderr)
        sys.exit(1)

    print(f"Ejecutando notebook para importar resultados → {nb_path}")
    ns = ejecutar_notebook_y_extraer_vars(nb_path)
    print("✔ Notebook ejecutado. Extrayendo matrices X,y…")
    ns_data = extraer_Xy_desde_namespace(ns)

    if args.modelo == "mlp":
        entrenar_mlp(ns_data, epochs=args.epochs, batch_size=args.batch_size)
    else:
        entrenar_cnn(ns_data, window=args.window, horizon=args.horizon,
                     epochs=args.epochs, batch_size=args.batch_size)


In [16]:
from pathlib import Path

NB_PATH = Path("feature_eng.ipynb")
WINDOW = 32
HORIZON = 1
EPOCHS = 200
BATCH_SIZE = 256


In [17]:
ns = ejecutar_notebook_y_extraer_vars(NB_PATH)
data = extraer_Xy_desde_namespace(ns)
print("Listo: X,y extraídos. Clases:", data.clases_unicas)

ModuleNotFoundError: No module named 'nbformat'

In [ ]:
mlp = entrenar_mlp(data, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [ ]:
cnn = entrenar_cnn(data, window=WINDOW, horizon=HORIZON, epochs=EPOCHS, batch_size=BATCH_SIZE)